In [1]:
from data_structures import Signal, Portfolio

ModuleNotFoundError: No module named 'data_structures'

In [ ]:
# set up
temp = df[46:49]
my_portfolio = Portfolio(5)

# buy signals
my_portfolio.buyFromDf(temp, 1)

# check that they have been bought
print('\nAFTER FIRST BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))


# update price and value
my_portfolio.updateMonthlyPrice()
my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER ONE PRICES AND VALUES UPDATE')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# update price and value until expiry
for i in range(153):
    my_portfolio.updateMonthlyPrice()
    my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER MANY PRICES AND VALUES UPDATES')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# dump expired
my_portfolio.dumpExpired()

# check that expired signals have been sold at last available price
print('\nAFTER EXPIRED DUMPED')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# buy more signals
temp = df[49:50]
my_portfolio.buyFromDf(temp, 4)

# check that cash is gone and oldest stock has been replaced
print('\nAFTER ANOTHER BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))